1. Configuración Inicial y Librerías
Explicación:
Importamos las librerías necesarias para el análisis y procesamiento de datos
pandas: Para manipulación de datos
numpy: Para operaciones numéricas
seaborn y matplotlib: Para visualizaciones
datetime: Para manejo de fechas
Configuramos el estilo de las gráficas para mejor visualización

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

plt.style.use('default')
sns.set_theme()
plt.rcParams['figure.figsize'] = (12, 8)

2. Carga de Datos
Explicación:
Cargamos el dataset desde el archivo CSV
Mostramos las primeras 5 filas para verificar la carga correcta

In [ ]:
df = pd.read_csv('../data/chatbot_satisfaction_dataset_utf8.csv')
df.head()

3. Ingeniería de Características Temporales
Explicación:
Convertimos la columna de fecha a formato datetime para poder extraer componentes temporales
Extraemos el año de la consulta
Extraemos el mes de la consulta
Extraemos el día de la semana (0-6, donde 0 es lunes)
Extraemos el trimestre del año
Creamos una variable binaria que indica si la consulta fue en fin de semana

In [ ]:
# Convertir fecha_consulta a datetime
df['fecha_consulta'] = pd.to_datetime(df['fecha_consulta'])

# Extraer características temporales
df['año_consulta'] = df['fecha_consulta'].dt.year
df['mes_consulta'] = df['fecha_consulta'].dt.month
df['dia_semana'] = df['fecha_consulta'].dt.dayofweek
df['trimestre'] = df['fecha_consulta'].dt.quarter

# Crear variable binaria para fin de semana
df['es_fin_semana'] = df['dia_semana'].isin([5, 6]).astype(int)

4. Ingeniería de Características de Tiempo de Servicio
   Explicación:
Calculamos el tiempo total de atención sumando tiempo de consulta y espera
Creamos un ratio que muestra la proporción entre tiempo de espera y tiempo de consulta
Categorizamos el tiempo de espera en 4 niveles usando cuartiles

In [ ]:
# Calcular tiempo total de atención
df['tiempo_total_atencion'] = df['tiempo_consulta'] + df['tiempo_espera']

# Calcular ratio de tiempo de espera vs tiempo de consulta
df['ratio_tiempo_espera_consulta'] = df['tiempo_espera'] / df['tiempo_consulta']

# Crear categorías de tiempo de espera
df['categoria_tiempo_espera'] = pd.qcut(df['tiempo_espera'], q=4, labels=['Muy Bajo', 'Bajo', 'Alto', 'Muy Alto'])

5. Ingeniería de Características de Calidad y Satisfacción
   Explicación:
Combinamos la calidad del servicio con la resolución de la consulta
Creamos un índice de satisfacción general promediando satisfacción e índice promotor
Calculamos la eficiencia del servicio basada en la resolución y el tiempo total

In [ ]:
# Crear variable combinada de calidad y resolución
df['calidad_resolucion'] = df['calidad_del_servicio'] * df['consulta_resuelta'].astype(int)

# Crear variable de satisfacción general
df['satisfaccion_general'] = (df['satisfaccion_estudiante'] + df['indice_promotor_neto']) / 2

# Crear variable de eficiencia del servicio
df['eficiencia_servicio'] = df['consulta_resuelta'].astype(int) / (df['tiempo_total_atencion'] + 1)

6. Codificación de Variables Categóricas
Explicación:
Convertimos variables categóricas a formato numérico
Usamos one-hot encoding para tipo de consulta y mes
Usamos codificación ordinal para urgencia (1=Baja, 2=Media, 3=Alta)

In [ ]:
# Codificación one-hot para tipo_consulta
tipo_consulta_dummies = pd.get_dummies(df['tipo_consulta'], prefix='tipo_consulta')
df = pd.concat([df, tipo_consulta_dummies], axis=1)

# Codificación ordinal para urgencia
urgencia_map = {'Baja': 1, 'Media': 2, 'Alta': 3}
df['urgencia_encoded'] = df['urgencia'].map(urgencia_map)

# Codificación one-hot para mes_consulta
mes_dummies = pd.get_dummies(df['mes_consulta'], prefix='mes')
df = pd.concat([df, mes_dummies], axis=1)

7. Creación de Características de Interacción
Explicación:
Creamos variables de interacción entre diferentes características
Estas interacciones pueden capturar relaciones no lineales entre variables

In [ ]:
# Interacción entre urgencia y tiempo de espera
df['urgencia_tiempo_espera'] = df['urgencia_encoded'] * df['tiempo_espera']

# Interacción entre calidad y tiempo de consulta
df['calidad_tiempo_consulta'] = df['calidad_del_servicio'] * df['tiempo_consulta']

# Interacción entre derivado y tiempo total
df['derivado_tiempo_total'] = df['derivado'].astype(int) * df['tiempo_total_atencion']

8. Normalización de Variables Numéricas
Explicación:
Normalizamos las variables numéricas para que estén en el rango [0,1]
Esto ayuda a que todas las variables tengan la misma escala
Mejora el rendimiento de los algoritmos de machine learning

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Seleccionar variables numéricas para normalizar
numeric_features = ['tiempo_consulta', 'tiempo_espera', 'tiempo_total_atencion', 
                   'ratio_tiempo_espera_consulta', 'eficiencia_servicio']

# Aplicar normalización Min-Max
scaler = MinMaxScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

9. Guardado del Dataset Procesado
Explicación:
Guardamos el dataset procesado con todas las nuevas características
El archivo se guarda en formato CSV
No incluimos el índice en el archivo guardado
¿Te gustaría que profundice en alguna de estas secciones o que te explique algún concepto específico con más detalle?

In [ ]:
# Guardar dataset procesado
df.to_csv('../data/chatbot_satisfaction_dataset_engineered.csv', index=False)
print("Dataset guardado exitosamente con las nuevas características")